In [1]:
from lib.datasets import ATPBind3D

# dataset_orig = ATPBind3D()
# dataset_orig.initialize_mask_and_weights()

In [7]:
dataset = ATPBind3D(to_slice=True)
dataset.initialize_mask_and_weights()

Initialize Undersampling: all ones
Initialize Weighting: all ones


ATPBind3D(
  #sample: 631
  #task: 1
)

In [35]:
from lib.pipeline import Pipeline
import pandas as pd
from torchdrug import utils, data
import os
import torch
import inspect
import numpy as np

from lib.utils import generate_mean_ensemble_metrics_auto, read_initial_csv, aggregate_pred_dataframe
GPU = 0

from atpbind_main import main


In [2]:
main(model_key='esm-t33', valid_fold=0)

batch_size: 8
init pipeline, model: esm-t33, dataset: atpbind3d, gpus: [0]
load model esm-t33, kwargs: {'gpu': 0, 'freeze_esm': False, 'freeze_layer_count': 30}
get dataset atpbind3d
Initialize Undersampling: all ones
Initialize Weighting: all ones
train samples: 302, valid samples: 76, test samples: 41
use cyclic lr scheduler
pipeline batch_size: 8
0m20s {'sensitivity': 0.5949, 'precision': 0.5902, 'mcc': 0.5702, 'micro_auprc': 0.6137, 'valid_mcc': 0.541, 'train_bce': 0.166, 'valid_bce': 0.1067, 'lr': 0.00084}
0m19s {'sensitivity': 0.6938, 'precision': 0.6601, 'mcc': 0.6586, 'micro_auprc': 0.7047, 'valid_mcc': 0.6246, 'train_bce': 0.0828, 'valid_bce': 0.0937, 'lr': 0.00138}
0m20s {'sensitivity': 0.6204, 'precision': 0.6946, 'mcc': 0.6389, 'micro_auprc': 0.6339, 'valid_mcc': 0.6264, 'train_bce': 0.0429, 'valid_bce': 0.1536, 'lr': 0.00192}
0m21s {'sensitivity': 0.6715, 'precision': 0.6651, 'mcc': 0.6501, 'micro_auprc': 0.6658, 'valid_mcc': 0.6298, 'train_bce': 0.0323, 'valid_bce': 0.124

In [1]:
from lib.pipeline import Pipeline
from torchdrug import utils, data
import torch
import numpy as np

CYCLE_SIZE = 10
pipeline = Pipeline(
    dataset='atpbind3d',
    model='lm-gearnet',
    gpus=[1],
    model_kwargs={
        'gpu': 1,
        'lm_type': 'esm-t33',
        'gearnet_hidden_dim_size': 512,
        'gearnet_hidden_dim_count': 4,
        'lm_freeze_layer_count': 30,
    },
    scheduler='cyclic',
    scheduler_kwargs={
        'base_lr': 3e-4,
        'max_lr': 3e-3,
        'step_size_up': CYCLE_SIZE / 2,
        'step_size_down': CYCLE_SIZE / 2,
        'cycle_momentum': False
    },
    batch_size=8,
    dataset_args={
        'to_slice': True,
        'max_slice_length': 650,
        'padding': 50,
    },
    valid_fold_num=0,
)

init pipeline, model: lm-gearnet, dataset: atpbind3d, gpus: [1]
load model lm-gearnet, kwargs: {'gpu': 1, 'lm_type': 'esm-t33', 'gearnet_hidden_dim_size': 512, 'gearnet_hidden_dim_count': 4, 'lm_freeze_layer_count': 30}
freeze_lm: 30
get dataset atpbind3d
Initialize Undersampling: all ones
Initialize Weighting: all ones
train samples: 332, valid samples: 84, test samples: 41
use cyclic lr scheduler
pipeline batch_size: 8


In [2]:
pipeline.train_until_fit(patience=10, max_epoch=10)

1m1s {'sensitivity': 0.5074, 'precision': 0.4252, 'mcc': 0.4363, 'micro_auprc': 0.4531, 'valid_mcc': 0.4378, 'train_bce': 0.1691, 'valid_bce': 0.1483, 'lr': 0.00084}
0m59s {'sensitivity': 0.5923, 'precision': 0.5958, 'mcc': 0.5745, 'micro_auprc': 0.5905, 'valid_mcc': 0.5859, 'train_bce': 0.0745, 'valid_bce': 0.1103, 'lr': 0.00138}
0m58s {'sensitivity': 0.6071, 'precision': 0.6559, 'mcc': 0.614, 'micro_auprc': 0.6314, 'valid_mcc': 0.6012, 'train_bce': 0.0469, 'valid_bce': 0.1081, 'lr': 0.00192}
0m59s {'sensitivity': 0.5565, 'precision': 0.6788, 'mcc': 0.598, 'micro_auprc': 0.5873, 'valid_mcc': 0.6143, 'train_bce': 0.0321, 'valid_bce': 0.1387, 'lr': 0.00246}
1m0s {'sensitivity': 0.6161, 'precision': 0.5061, 'mcc': 0.5349, 'micro_auprc': 0.5965, 'valid_mcc': 0.6186, 'train_bce': 0.0313, 'valid_bce': 0.1092, 'lr': 0.003}
1m1s {'sensitivity': 0.4301, 'precision': 0.7918, 'mcc': 0.5699, 'micro_auprc': 0.6209, 'valid_mcc': 0.6265, 'train_bce': 0.023, 'valid_bce': 0.1262, 'lr': 0.00246}
1m2s {

[{'sensitivity': 0.5074,
  'precision': 0.4252,
  'mcc': 0.4363,
  'micro_auprc': 0.4531,
  'valid_mcc': 0.4378,
  'train_bce': 0.1691,
  'valid_bce': 0.1483,
  'lr': 0.00084},
 {'sensitivity': 0.5923,
  'precision': 0.5958,
  'mcc': 0.5745,
  'micro_auprc': 0.5905,
  'valid_mcc': 0.5859,
  'train_bce': 0.0745,
  'valid_bce': 0.1103,
  'lr': 0.00138},
 {'sensitivity': 0.6071,
  'precision': 0.6559,
  'mcc': 0.614,
  'micro_auprc': 0.6314,
  'valid_mcc': 0.6012,
  'train_bce': 0.0469,
  'valid_bce': 0.1081,
  'lr': 0.00192},
 {'sensitivity': 0.5565,
  'precision': 0.6788,
  'mcc': 0.598,
  'micro_auprc': 0.5873,
  'valid_mcc': 0.6143,
  'train_bce': 0.0321,
  'valid_bce': 0.1387,
  'lr': 0.00246},
 {'sensitivity': 0.6161,
  'precision': 0.5061,
  'mcc': 0.5349,
  'micro_auprc': 0.5965,
  'valid_mcc': 0.6186,
  'train_bce': 0.0313,
  'valid_bce': 0.1092,
  'lr': 0.003},
 {'sensitivity': 0.4301,
  'precision': 0.7918,
  'mcc': 0.5699,
  'micro_auprc': 0.6209,
  'valid_mcc': 0.6265,
  'tra

In [1]:
from lib.datasets import get_slices, protein_to_slices
from lib.utils import dict_tensor_to_num, round_dict

dataset = pipeline.dataset

from torchdrug import transforms
pipeline.dataset.transform = transforms.ProteinView(view='residue')

train_set, valid_set, test_set = pipeline.dataset.split()

def infer_sliced(pipeline, protein, max_slice_length=550, padding=100):
    GPU = 1
    target = protein.target
    intermediate_preds = []
    sliced_proteins, _ = protein_to_slices(protein, target, max_slice_length=max_slice_length, padding=padding)
    dataloader = data.DataLoader(sliced_proteins, batch_size=1, shuffle=False)
    with torch.no_grad():
        for batch in dataloader:
            batch = utils.cuda(batch, device=torch.device(f'cuda:{GPU}'))
            pred = pipeline.task.predict({"graph": batch})
            intermediate_preds.append(pred)
    final_preds = torch.zeros(target.shape)
    for i, (start, end) in enumerate(get_slices(target.shape[0], max_slice_length=max_slice_length, padding=padding)):
        final_preds[start:end] += intermediate_preds[i].cpu()
        if i > 0:
            final_preds[start:start+padding] /= 2
    return final_preds



def get_pred_and_target(data_set):
    preds = []
    targets = []
    for item in data_set:
        pred = infer_sliced(pipeline, item['graph'])
        preds.append(pred)

        dataloader = data.DataLoader([item], batch_size=1, shuffle=False)    
        target = pipeline.task.target(next(iter(dataloader)))
        targets.append(target)
        
    pred = utils.cat(preds)
    target = utils.cat(targets)
    return pred, target


pred, target = get_pred_and_target(test_set)

print(pred)

metric = pipeline.task.evaluate(pred, target, threshold=0)
round_dict(dict_tensor_to_num(metric), 4)

NameError: name 'pipeline' is not defined

In [4]:
metric = pipeline.task.evaluate(pred, target, threshold=-3)
round_dict(dict_tensor_to_num(metric), 4)

{'sensitivity': 0.6424,
 'precision': 0.7193,
 'mcc': 0.6654,
 'micro_auprc': 0.667}

In [5]:
def get_best_mcc(pipeline, valid_set):
    pred, target = get_pred_and_target(valid_set)
    thresholds = np.linspace(-4, 0, num=41)
    mcc_values = [pipeline.task.evaluate(pred, target, threshold=th)['mcc'] for th in thresholds]
    
    max_mcc_idx = np.argmax(mcc_values)
    print(mcc_values)
    return round(thresholds[max_mcc_idx], 2), mcc_values[max_mcc_idx]


get_best_mcc(pipeline, valid_set)

[0.6646703454160542, 0.662160158839212, 0.6612423519012024, 0.6609006221576132, 0.662631888129046, 0.6633440897293058, 0.6609656771780936, 0.6615542827514305, 0.662292589338338, 0.6646785832673293, 0.6654471338851848, 0.6639765149700037, 0.6629371748530145, 0.6599839387737801, 0.6601611596247207, 0.6628192745832914, 0.6623962673658689, 0.6605035113758178, 0.6619764471630109, 0.6609258404606511, 0.6621974831143543, 0.6616050602318791, 0.6635467778801908, 0.6637966016312562, 0.6633967444009901, 0.6636560757280595, 0.6663065592044322, 0.6655319605896771, 0.6668761294871491, 0.6643781664055507, 0.6656655207514386, 0.6649196576071046, 0.6670010722062665, 0.6645088344438979, 0.6623756303164436, 0.6616430052226454, 0.6616264425799427, 0.6601939905847326, 0.6601939905847326, 0.660909513699043, 0.6616271166235279]


(-0.8, 0.6670010722062665)

In [6]:
metric = pipeline.task.evaluate(pred, target, threshold=-0.8)
round_dict(dict_tensor_to_num(metric), 4)

{'sensitivity': 0.5979,
 'precision': 0.7735,
 'mcc': 0.667,
 'micro_auprc': 0.667}